# À la façon d'Andy Warhol - 2

L'activité suivante complète l'activité de seconde située dans le même dossier.

Il peut être nécessaire de revoir cette activité avant d'aborder celle-ci :

On va disposer côte-à-côté les trois images issus des canaux RVB de l'image.

La commande `display` permet l'affichage d'une image mais pour envisager un affichage côte-à-côte de plusiseurs images, on doit envisager l'utilisation de `widgets` (le mot `widget` est une contraction de **WI**ndow ga**DGET**) spécifiques à l'interface de Jupyter : ce sont de petits modules graphiques indépendants (champs de saisie, boutons, etc. [voir la doc des widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)).

On crée donc un widget par image et on les place dans une `HBOX` : un conteneur qui empile horizontalement.

*Problème* les objets de type `Image` créés par `pillow` ne conviennent pas pour ces widgets qui n'acceptent qu'un format binaire (les octets issus de l'image). Une petite manipulation est donc nécessaire pour les convertir.

*Nota Bene* dans ce code, on ne sauvegarde plus les images créées.

On commence par importer toutes les bibliothèques nécessaires :

In [ ]:
# installation de pillow
import sys
!{sys.executable} -m pip install pillow
# puis importation du module Image
from PIL import Image
# utilisation de widgets pour une disposition en ligne des images
import ipywidgets as widgets
# utilisation de io pour conversion des images
import io

On définit l'objet Image pillow avec lequel on va travailler dans la suite :

In [ ]:
fichier_image = 'images/boole.jpg'
img = Image.open(fichier_image)

largeur, hauteur = img.size
print('Nombre de pixels en largeur :', largeur)
print('Nombre de pixels en hauteur :', hauteur)

Pour disposer côte-à-côte, on les crée avec la même variable `nouvelle_img`. Après chaque parcours de tous les pixels, on convertit l'ensemble obtenu en suite d'octets puis en widget de type image.

Chaque widget est ensuite ajouté à la liste des widgets. La liste est affichée ensuite dans un widget de type conteneur horizontal.

In [ ]:
# liste des images sous forme de widgets
wid = []

# création de trois images -> les 3 canaux RVB de l'image
for i in range(3):
    # on crée la nouvelle image
    nouvelle_img = Image.new("RGB", img.size)
    for x in range(largeur):
        for y in range(hauteur):
            # on choisit le canal
            c = img.getpixel((x, y))[i]
            couleur = [0, 0, 0]
            couleur[i] = c
            # putpixel n'accepte que les entiers ou les tuples
            nouvelle_img.putpixel((x, y), tuple(couleur))
    # transformation des images en format accepté par widgets.Image
    # (en binaire)
    img_byte_arr = io.BytesIO()
    nouvelle_img.save(img_byte_arr, format='jpeg')
    img_byte_arr = img_byte_arr.getvalue()
    # création d'un widget et ajout dans la liste des widgets
    wid.append(widgets.Image(value=img_byte_arr))

# arrangement des widgets
boite = widgets.HBox(wid)
# affichage
display(boite)

Bien sûr pour diminuer le coût, on peut engendrer les trois images dans la même boucle. On différencie alors les trois images :

In [ ]:
rouge = Image.new("RGB", img.size)
vert = Image.new("RGB", img.size)
bleu = Image.new("RGB", img.size)

for x in range(largeur):
    for y in range(hauteur):
        R, V, B = img.getpixel((x, y))
        rouge.putpixel((x, y), (R, 0, 0))
        vert.putpixel((x, y), (0, V, 0))
        bleu.putpixel((x, y), (0, 0, B))

wid = []

for image in [rouge, vert, bleu]:
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='jpeg')
    img_byte_arr = img_byte_arr.getvalue()
    wid.append(widgets.Image(value=img_byte_arr))

    
boite = widgets.HBox(wid)
display(boite)

Cette différenciation par un nom empêche toute évolution du code. Ainsi, on peut opter pour une stratégie différente : déclarer les besoins. On utilise alors une structure déclarant les canaux à obtenir pour chaque image. Une liste de dimension 3 précise trois images à obtenir, par exemple avec les 3 images précédentes qui expriment le triplet R (canal 0), V (canal 1) et B (canal 2), on prendrait :

```python
canaux_voulus = [[0], [1], [2]]
```

On pourrait pour chaque image demander l'expression de plusieurs canaux :

```python
canaux_voulus = [[0], [1], [0, 1]]
```

La dernière image exprimera le rouge et le vert donc aura une dominante jaune.

In [ ]:
canaux_voulus = [[0], [1], [0, 1]]

images = [Image.new('RGB', img.size) for _ in range(len(canaux_voulus))]

for x in range(largeur):
    for y in range(hauteur):
        R, V, B = img.getpixel((x, y))
        # enumerate permet d'obtenir une numérotation qui sera
        # donnée par i pour la boucle
        for i, im in enumerate(images):
            couleur = [0, 0, 0]
            # on parcourt les canaux voulus pour cette image
            for indice in canaux_voulus[i]:
                couleur[indice] = img.getpixel((x, y))[indice]
            im.putpixel((x, y), tuple(couleur))

wid = []

for image in images:
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='jpeg')
    img_byte_arr = img_byte_arr.getvalue()
    wid.append(widgets.Image(value=img_byte_arr))

boite = widgets.HBox(wid)
display(boite)

On termine avec un affichage d'images en 3 x 3 (3 en largeur et 3 en hauteur).

Sur le même pricipe vue avec le widget `HBox`, on peut empiler verticalement avec un widget `VBox`.

In [ ]:
canaux_voulus = [[0], [1], [0, 1], [1], [2], [1, 2], [2], [0], [2, 0]]

images = [Image.new('RGB', img.size) for _ in range(len(canaux_voulus))]

for x in range(largeur):
    for y in range(hauteur):
        R, V, B = img.getpixel((x, y))
        for i, im in enumerate(images):
            couleur = [0, 0, 0]
            for indice in canaux_voulus[i]:
                couleur[indice] = img.getpixel((x, y))[indice]
            im.putpixel((x, y), tuple(couleur))

wid = []

for i, image in enumerate(images):
    # on crée le conteneur horizontal
    if i % 3 == 0:
        hwid = []
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='jpeg')
    img_byte_arr = img_byte_arr.getvalue()
    hwid.append(widgets.Image(value=img_byte_arr))
    # fin du widget horizontal, on l'ajoute à la liste
    # pour le widget vertical
    if (i + 1) % 3 == 0:
        wid.append(widgets.HBox(hwid))

boite = widgets.VBox(wid)
display(boite)